# BETTER WAY 17. 인수를 순회할 때는 방어적으로 하자
1. 그냥
2. 입력 이터레이터를 방어적으로 복사하기
3. lambda 표현식 사용하기
4. Iterator protocol을 구현한 새 컨테이너 클래스 이용하기

In [4]:
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / float(total)
        result.append(percent)
    return result

visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [13]:
def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)

def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / float(total)
        result.append(percent)
    return result

it = read_visits("./tmp/my_numbers.txt")
percentages = normalize(it)
print(percentages)

[]


In [8]:
it = read_visits("./tmp/my_numbers.txt")
print(list(it))
print(list(it))

[15, 35, 80]
[]


### ▶︎ 입력 이터레이터를 방어적으로 복사하기
- 단, 입력받은 이터레이터 콘텐츠의 복사본이 클 경우엔 메모리 고갈이 우려됨

In [14]:
def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)

def normalize(numbers):
    numbers = list(numbers) #이터레이터를 복사함
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / float(total)
        result.append(percent)
    return result

it = read_visits("./tmp/my_numbers.txt")
percentages = normalize(it)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


### ▶︎ lambda 표현식 사용하기
- 함수를 딱 한 줄만으로 표현 가능
- 'lambda 인자 : 표현식' 으로 사용가능
- lambda 예시
- ~~~
 def add(x,y):
    return x + y

 add(10, 20)~~~
- 위와 아래의 표현은 같은 의미
- ~~~
 (lambda x,y: x + y)(10, 20)~~~

In [19]:
it = lambda: read_visits("./tmp/my_numbers.txt")
print(list(it()))
print(list(it()))

[15, 35, 80]
[15, 35, 80]


In [26]:
def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)

def normalize_func(get_iter):
    total = sum(get_iter()) # 새 이터레이터
    result = []
    for value in get_iter(): # 새 이터레이터
        percent = 100 * value / float(total)
        result.append(percent)
    return result

path = "./tmp/my_numbers.txt"
percentages = normalize_func(lambda: read_visits(path)) # 이러한 방법은 세련되지 못하긴 하다.
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


### ▶︎ Iterator protocol을 구현한 새 컨테이너 클래스 이용하기
- 이 방법의 유일한 단점은 입력 데이터를 여러 번 읽는다는 점이다.

In [49]:
class ReadVisits(object):
    def __init__(self, data_path):
        self.data_path = data_path
        
    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

def normalize(numbers):
    numbers = list(numbers) #이터레이터를 복사함
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / float(total)
        result.append(percent)
    return result

path = "./tmp/my_numbers.txt"
visits = ReadVisits(path)
percentages = normalize(visits)
print(percentages)

print(visits)
print(visits.__iter__)

[11.538461538461538, 26.923076923076923, 61.53846153846154]
<bound method ReadVisits.__iter__ of <__main__.ReadVisits object at 0x10467f7d0>>


### ▶︎ 한 번 이터레이션을 돌고나면 끝나버리는 단순한 이터레이터를 걸러주기

In [55]:
class ReadVisits(object):
    def __init__(self, data_path):
        self.data_path = data_path
        
    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

def normalize_defensive(numbers):
    if iter(numbers) is iter(numbers):
        raise TypeError("Must supply a container")
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / float(total)
        result.append(percent)
    return result


visits = [15, 35, 80]
percentages1 = normalize_defensive(visits)

path = "./tmp/my_numbers.txt"
visits = ReadVisits(path)
percentages2 = normalize_defensive(visits)

print(percentages1)
print(percentages2)

[11.538461538461538, 26.923076923076923, 61.53846153846154]
[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [78]:
it = iter(visits)
print iter(visits) is iter(visits)
print iter(it) is iter(it)
print sum(iter(visits)), sum(iter(visits))
print sum(iter(it)),sum(iter(it))
normalize_defensive(it)

False
True
130 130
130 0


TypeError: Must supply a container